In [17]:
import re
import pandas as pd

log_entries = """
Jun  6 07:35:12 interswitch-virtual-machine haproxy[3732]: 41.210.186.148:51732 [06/Jun/2023:07:35:12.147] www-https~ interswitchstore_backend/interswitchstore_webserver 371/0/1/37/409 200 217 - - ---- 452/452/0/1/0 0/0 "POST /interswitchstore/device/remoteConfigs HTTP/1.1"
Jun  6 07:35:12 interswitch-virtual-machine haproxy[3732]: 196.13.203.201:59225 [06/Jun/2023:07:35:12.378] www-https~ api_backend/api_webserver2 3/0/1/187/191 200 314 - - ---- 452/452/0/1/0 0/0 "POST /api/v1A/svapayments/validateCustomer HTTP/1.1"
Jun  6 07:35:12 interswitch-virtual-machine haproxy[3732]: 197.239.10.226:1517 [06/Jun/2023:07:35:12.612] www-https~ interswitchstore_backend/interswitchstore_webserver 121/0/1/51/173 500 2410 - - ---- 452/452/0/0/0 0/0 "POST /interswitchstore/device/geolocationstatus HTTP/1.1"
Jun  6 07:35:12 interswitch-virtual-machine haproxy[3732]: 196.13.203.201:59225 [06/Jun/2023:07:35:12.569] www-https~ api_backend/api_webserver1 4/0/0/242/246 200 314 - - ---- 452/452/0/1/0 0/0 "POST /api/v1A/svapayments/validateCustomer HTTP/1.1"
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 196.13.203.201:59225 [06/Jun/2023:07:35:12.814] www-https~ api_backend/api_webserver2 29/0/1/207/237 200 314 - - ---- 452/452/0/1/0 0/0 "POST /api/v1A/svapayments/validateCustomer HTTP/1.1"
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 196.13.203.201:59225 [06/Jun/2023:07:35:13.052] www-https~ api_backend/api_webserver1 4/0/0/223/227 200 314 - - ---- 452/452/0/1/0 0/0 "POST /api/v1A/svapayments/validateCustomer HTTP/1.1"
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 196.13.203.201:59225 [06/Jun/2023:07:35:13.280] www-https~ api_backend/api_webserver2 20/0/1/218/239 200 314 - - ---- 452/452/0/1/0 0/0 "POST /api/v1A/svapayments/validateCustomer HTTP/1.1"
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 41.210.159.101:46487 [06/Jun/2023:07:35:13.433] www-https/1: SSL handshake failure
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 41.210.159.101:46488 [06/Jun/2023:07:35:13.560] www-https/1: SSL handshake failure
Jun  6 07:35:13 interswitch-virtual-machine haproxy[3732]: 41.210.159.101:46489 [06/Jun/2023:07:35:13.560] www-https/1: SSL handshake failure
"""

# Regular expression pattern
pattern = r'^(.*?)\s+\S+\s+(\S+):\d+\s+\[(.*?)\]\s+(.*?)\s+(\S+)\s+(.*?)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S+)\s+"(.*?)"$'

# Parse the log entries using the regular expression
parsed_entries = re.findall(pattern, log_entries, re.MULTILINE)

# Parse the log entries using the regular expression
parsed_logs = pd.DataFrame([re.findall(pattern, entry)[0] for entry in log_entries.split('\n') if re.findall(pattern, entry)],
                           columns=['timestamp', 'client_ip_port', 'received_timestamp', 'frontend', 'backend', 'process',
                                    'concurrent_connections', 'frontend_queue', 'backend_active', 'server_queue_limit',
                                    'queue_time', 'http_status_code', 'bytes_received', 'termination_state',
                                    'wait_connect_time', 'wait_backend_time', 'wait_queue_time', 'backend_connect_time',
                                    'process_time', 'active_connections', 'active_requests', 'http_method_path_version'])

# Print the parsed entries
#for entry in parsed_entries:
#    print(entry)


('Jun  6 07:35:12 interswitch-virtual-machine', '41.210.186.148', '06/Jun/2023:07:35:12.147', 'www-https~', 'interswitchstore_backend/interswitchstore_webserver', '371/0/1/37/409 200', '217', '-', '-', '----', '452/452/0/1/0', '0/0', 'POST /interswitchstore/device/remoteConfigs HTTP/1.1')
('Jun  6 07:35:12 interswitch-virtual-machine', '196.13.203.201', '06/Jun/2023:07:35:12.378', 'www-https~', 'api_backend/api_webserver2', '3/0/1/187/191 200', '314', '-', '-', '----', '452/452/0/1/0', '0/0', 'POST /api/v1A/svapayments/validateCustomer HTTP/1.1')
('Jun  6 07:35:12 interswitch-virtual-machine', '197.239.10.226', '06/Jun/2023:07:35:12.612', 'www-https~', 'interswitchstore_backend/interswitchstore_webserver', '121/0/1/51/173 500', '2410', '-', '-', '----', '452/452/0/0/0', '0/0', 'POST /interswitchstore/device/geolocationstatus HTTP/1.1')
('Jun  6 07:35:12 interswitch-virtual-machine', '196.13.203.201', '06/Jun/2023:07:35:12.569', 'www-https~', 'api_backend/api_webserver1', '4/0/0/242/246